In [133]:
import pandas as pd
import doctest
import numpy as np
from sklearn_pandas import DataFrameMapper

In [87]:
from sklearn.pipeline import (
    Pipeline,
    FeatureUnion,
)

from sklearn.compose import (
    ColumnTransformer,
    make_column_transformer,
)

from sklearn.preprocessing import (
    FunctionTransformer,
    OneHotEncoder,
)

In [88]:
raw_data = pd.read_csv('data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_1.csv')

/Users/merickson/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (48,69,76,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [97]:
def convert_time_to_hour(minutes_since_midnight):
    """Returns the whole number of hours since midnight, roundeded down.
    
    Examples:
    >>> convert_time_to_hour(33)
    0
    >>> convert_time_to_hour(1353)
    22
    """
    return minutes_since_midnight // 60

In [98]:
doctest.testmod()

TestResults(failed=0, attempted=2)

Pipeline steps:
    * Convert DepTime and ArrTime to hour
    * One-Hot Encode Everything

In [99]:
target_col = 'DepDelayMinutes'

In [125]:
predictor_cols = [
    'DepTime',
    'ArrTime',
    'DayOfWeek',
    'OriginAirportID',
    'DestAirportID',
    'Reporting_Airline',
]

In [126]:
cols = predictor_cols + [target_col]

In [127]:
data = raw_data.loc[:, cols].dropna()

In [128]:
predictor_col_idxs = [data.columns.get_loc(c) for c in predictor_cols]

In [129]:
select_columns = ColumnTransformer(
    remainder='drop',
    transformers=[
        ('select_columns', 'passthrough', predictor_col_idxs),
    ])

In [130]:
preprocess_times = ColumnTransformer(
    remainder='drop',
    transformers=[
        ('convert_time_to_hour', FunctionTransformer(convert_time_to_hour), [0, 1]),
        ('keep_other_columns', 'passthrough', [2, 3, 4, 5]),
    ],

)

In [131]:
pipe = Pipeline([
    ('select columns', select_columns),
    ('preprocess time fields', preprocess_times),
    #('one-hot encode everything', OneHotEncoder),
])

In [132]:
pipe.fit(data)

/Users/merickson/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/Users/merickson/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('select columns', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('select_columns', 'passthrough', [0, 1, 2, 3, 4, 5])])), ('preprocess time fields', ColumnTransformer(n_jobs=None, remainder='drop', sparse_thre...ass_y='deprecated', validate=None), [0, 1]), ('keep_other_columns', 'passthrough', [2, 3, 4, 5])]))])

In [ ]:
pipe.

In [124]:
pipe.transform(data)

/Users/merickson/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


array([[22.0, 24.0, 5, 14698, 14771, 'OO'],
       [15.0, 18.0, 5, 12266, 15919, 'OO'],
       [10.0, 13.0, 5, 14783, 12266, 'OO'],
       ...,
       [20.0, 22.0, 3, 12173, 13830, 'HA'],
       [31.0, 33.0, 7, 12758, 12173, 'HA'],
       [29.0, 30.0, 7, 12173, 12758, 'HA']], dtype=object)

In [ ]:
mapper = DataFrameMapper([
    ()
])